# Visualization Example

In [54]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
import sys
DIR = "/workspace/nflbigdatabowl2023"
sys.path.append(DIR)

In [56]:
import math
from ast import literal_eval

import pandas as pd
import matplotlib.pyplot as plt

from src.pipeline.flows.main import main_flow
from src.visualization.interactive_play_selector import create_interactive_play_selector

In [57]:
_ = main_flow(max_weeks=3, max_games=3, max_plays=3)

05:11:01.385 | INFO    | prefect.engine - Created flow run 'aloof-labrador' for flow 'main-flow'

05:11:01.581 | INFO    | Flow run 'aloof-labrador' - Created task run 'read_csv-3609c996-0' for task 'read_csv'

05:11:01.585 | INFO    | Flow run 'aloof-labrador' - Executing 'read_csv-3609c996-0' immediately...

05:11:01.872 | INFO    | Task run 'read_csv-3609c996-0' - Finished in state Completed()

05:11:01.917 | INFO    | Flow run 'aloof-labrador' - Created task run 'read_csv-3609c996-1' for task 'read_csv'

...

05:11:18.928 | INFO    | Task run 'write_csv-386fe2af-4' - Finished in state Completed()

05:11:18.984 | INFO    | Flow run 'aloof-labrador' - Created task run 'write_csv-386fe2af-5' for task 'write_csv'

05:11:18.987 | INFO    | Flow run 'aloof-labrador' - Executing 'write_csv-386fe2af-5' immediately...

05:11:19.136 | INFO    | Task run 'write_csv-386fe2af-5' - Finished in state Completed()

05:11:19.209 | INFO    | Flow run 'aloof-labrador' - Finished in state Completed('All states completed.')

In [58]:
df_plays_all = pd.read_csv(f"{DIR}/data/raw/plays.csv")
df_tracking_display_all = pd.read_csv(f"{DIR}/data/outputs/tracking_display.csv")
df_areas_all = pd.read_csv(f"{DIR}/data/outputs/pocket_areas.csv")
df_areas_all["pocket"] = df_areas_all["pocket"].apply(literal_eval)

In [59]:
create_interactive_play_selector(
    df_plays_all,
    df_tracking_display_all,
    df_areas_all,
    continuous_update=True
)

interactive(children=(Dropdown(description='Week', options=(1, 2, 3), value=1), Dropdown(description='Game ID'…